# LOAD LIBRARIES

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(OPENAI_API_KEY)
# This is the YouTube video we're going to use.
YOUTUBE_VIDEO = "YOUTuBE_VIDEO"

MP3_FILE = "data/mp3s/MP3_OUTPUT_FILE.mp3"

gpt_version = "gpt-4o-mini"

# Get Transcription

A partir de aquí viene el código de la app:

In [ ]:
import tempfile
import whisper
from pytube import YouTube
from urllib.error import HTTPError

data_to_read = "data/transcription.txt"

# to solve HTTPError
from pytube. innertube import _default_clients
_default_clients[ "ANDROID"][ "context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients[ "ANDROID_EMBED"][ "context"][ "client"]["clientVersion"] = "19.08.35"
_default_clients[ "IOS_EMBED"][ "context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS_MUSIC"][ "context"]["client"]["clientVersion"] = "6.41"
_default_clients[ "ANDROID_MUSIC"] = _default_clients[ "ANDROID_CREATOR" ]


# Let's do this only if we haven't created the transcription file yet.
if not os.path.exists(data_to_read):
    file = None
    print("looking for mp3 file...")
    if not os.path.exists(MP3_FILE):
        print("loading youtube function")
        youtube = YouTube(YOUTUBE_VIDEO)
        
        print("getting audio path")
        try:
            audio = youtube.streams.filter(only_audio=True).first()
        except HTTPError:
            print("error downloading")
            audio = youtube.streams.filter(only_audio=True).first()

        print(f"Audio {audio}")
        
        with tempfile.TemporaryDirectory() as tmpdir:
            print("downloading audio")
            audio_file = audio.download(output_path=tmpdir)
       
    else:
        print(f"audio file exists: {MP3_FILE}")
        
        audio_file = MP3_FILE
 
 
    print("loading whisper model")
    whisper_model = whisper.load_model("large-v2")
        
    print("initiating transcription")
    transcription = whisper_model.transcribe(audio_file, fp16=False)["text"].strip()

    with open(data_to_read, "w") as file:
        file.write(transcription)

In [ ]:
with open(data_to_read) as file:
    transcription = file.read()

transcription[:100]

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=gpt_version)

template = """
Responda a la pregunta basándose en el contexto que aparece a continuación. 
Se trata de una entrevista de un podcast  
a un asesor financiero experto en seguros y productos de inversión.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model # | parser

question = """
Extraiga los puntos clave del podcast:
    Puntos a extraer:
    1. Resuma los principales temas tratados en el podcast.
    2. Identifique cualquier idea, hecho u opinión importante que se mencione.
    3. Destaque los argumentos o puntos de vista clave presentados por los oradores.
    4. Presente un resumen detallado de los principales temas tratados en el podcast.
    
    Genere un resumen detallado del contenido del podcast, centrándose en los puntos clave mencionados anteriormente. 
    
    Incluya suficiente información para captar la esencia de la discusión sin resumirla demasiado.
    
    """
    
question = """
    Qué tipo de productos son los que recomienda el entrevistado para invertir? 
    Da detalle de un par de párrafos de cada uno y la información que tengas de ellos.
    """
    
response = chain.invoke({
    "context": transcription,
    # "question": "can you summarize the podcast?"
    "question": question
})

response.pretty_print()